In [1]:
from operator import neg
import pandas as pd
import pickle
import requests
from requests.structures import CaseInsensitiveDict
import json
import shutil
import time

In [2]:
client_id = ""
client_secret = ""

In [3]:
# Get a bearer token.
petfinder_token_url = "https://api.petfinder.com/v2/oauth2/token"

data = {'grant_type' : 'client_credentials', 'client_id' : client_id , 'client_secret' : client_secret}

response = requests.post(petfinder_token_url, data = data)

json_response = json.loads(response.content)

bearer_token = json_response['access_token']

In [4]:
# Get Best Friends Society Organization.
petfinder_get_organizations_url = "https://api.petfinder.com/v2/organizations"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

params = {'name' : 'Best Friends Animal Society'}

response = requests.get(petfinder_get_organizations_url, params = params, headers = headers)

json_response = json.loads(response.content)

for i in range(5):
    print(json_response['organizations'][i])

{'id': 'UT81', 'name': 'Best Friends Animal Society-Utah', 'email': 'utahadoptions@bestfriends.org', 'phone': '(801) 574-2454', 'address': {'address1': '2005 S 1100 E', 'address2': None, 'city': 'Salt Lake City', 'state': 'UT', 'postcode': '84106', 'country': 'US'}, 'hours': {'monday': '10 am - 7 pm', 'tuesday': '10 am - 7 pm', 'wednesday': '10 am - 7 pm', 'thursday': '10 am - 7 pm', 'friday': '10 am - 7 pm', 'saturday': '10 am - 7 pm', 'sunday': '10 am - 7 pm'}, 'url': 'https://www.petfinder.com/member/us/ut/salt-lake-city/best-friends-animal-society-utah-ut81/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'website': 'http://utah.bestfriends.org', 'mission_statement': 'The mission of Best Friends Animal Society is to bring about a time when there are No More Homeless Pets....', 'adoption': {'policy': 'Due to Covid-19, our SLC Lifesaving Center is temporarily closed. All pets are currently in foster homes and adoption meet...', 'url': 'https://utah.bestfriends.org/get-involved/ado

In [5]:
# The Best Friends Animal Society's main office is the one with id "UT77".
# Getting 100 profiles from adoptable animals from that organization.
petfinder_get_animals_url = "https://api.petfinder.com/v2/animals"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

organization_id = "UT81"
limit = 100

params = {'organization' : organization_id, 'limit' : limit}

response = requests.get(petfinder_get_animals_url, params = params, headers = headers)

json_response = json.loads(response.content)

print(json_response)

{'animals': [{'id': 59054224, 'organization_id': 'UT81', 'url': 'https://www.petfinder.com/cat/mister-foo-59054224/ut/salt-lake-city/best-friends-animal-society-utah-ut81/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'type': 'Cat', 'species': 'Cat', 'breeds': {'primary': 'Domestic Short Hair', 'secondary': None, 'mixed': False, 'unknown': False}, 'colors': {'primary': 'Brown / Chocolate', 'secondary': None, 'tertiary': None}, 'age': 'Adult', 'gender': 'Male', 'size': 'Large', 'coat': None, 'attributes': {'spayed_neutered': True, 'house_trained': False, 'declawed': False, 'special_needs': False, 'shots_current': True}, 'environment': {'children': None, 'dogs': None, 'cats': None}, 'tags': [], 'name': 'Mister Foo', 'description': None, 'organization_animal_id': 'BSLC-A-22302', 'photos': [], 'primary_photo_cropped': None, 'videos': [], 'status': 'adoptable', 'status_changed_at': '2022-12-05T02:54:31+0000', 'published_at': '2022-12-05T02:54:30+0000', 'distance': None, 'contact': {'em

In [6]:
print(len(json_response['animals']))

97


In [7]:
# Harmonize data.
animals = json_response['animals']
harmonized_animals = []

for animal in animals:

    harmonized_animal = {'Type' : animal['type'], 'Name' : animal['name'], 'Age' : animal['age'],\
                           'Breed1' : animal['breeds']['primary'], 'Breed2' : animal['breeds']['secondary'], \
                           'Gender' : animal['gender'], 'Color1' : animal['colors']['primary'], \
                           'Color2' : animal['colors']['secondary'], 'Color3' : animal['colors']['tertiary'], \
                           'MaturitySize' : animal['size'], 'FurLength' : None, 'Vaccinated' : animal['attributes']['shots_current'], \
                           'Dewormed' : None, 'Sterilized' : animal['attributes']['spayed_neutered'], 'Health' : None, \
                           'Quantity' : 1, 'Fee' : None, 'State' : animal['contact']['address']['state'], 'RescuerID' : None, \
                           'VideoAmt' : len(animal['videos']), 'Description' : animal['description'], 'PetID' : animal['id'], \
                           'PhotoAmt' : len(animal['photos']), 'AdoptionSpeed' : None, 'id' : animal['id'] }
    harmonized_animals.append(harmonized_animal)

print(harmonized_animals)

[{'Type': 'Cat', 'Name': 'Mister Foo', 'Age': 'Adult', 'Breed1': 'Domestic Short Hair', 'Breed2': None, 'Gender': 'Male', 'Color1': 'Brown / Chocolate', 'Color2': None, 'Color3': None, 'MaturitySize': 'Large', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': True, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 59054224, 'PhotoAmt': 0, 'AdoptionSpeed': None, 'id': 59054224}, {'Type': 'Cat', 'Name': 'Dough Boy', 'Age': 'Adult', 'Breed1': 'Domestic Short Hair', 'Breed2': None, 'Gender': 'Male', 'Color1': 'Black', 'Color2': 'White', 'Color3': None, 'MaturitySize': 'Large', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': True, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': 'My name is Dough Boy and I love people and food! I love to cuddle and talk to humans but...', 'PetID': 59046524, 'PhotoAmt': 4, 'AdoptionSpe

In [8]:
df = pd.DataFrame(harmonized_animals)

In [9]:
df.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,id
0,Cat,Mister Foo,Adult,Domestic Short Hair,None,Male,Brown / Chocolate,None,None,Large,...,1,None,UT,None,0,None,59054224,0,None,59054224
1,Cat,Dough Boy,Adult,Domestic Short Hair,None,Male,Black,White,None,Large,...,1,None,UT,None,0,My name is Dough Boy and I love people and foo...,59046524,4,None,59046524
2,Cat,Candy Corn,Adult,Domestic Short Hair,None,Male,Brown / Chocolate,None,None,Large,...,1,None,UT,None,0,Say hello to Candy Corn! This sweet boy has al...,59045778,2,None,59045778
3,Cat,Tots,Adult,Domestic Short Hair,None,Male,Black,White,None,Medium,...,1,None,UT,None,0,"Hello humans, my name is Tots! I am a gentle b...",59045779,5,None,59045779
4,Cat,Twinkie,Baby,Domestic Short Hair,None,Male,Brown / Chocolate,None,None,Medium,...,1,None,UT,None,0,None,59044794,1,None,59044794
5,Cat,Twootsie Roll,Baby,Domestic Short Hair,None,Female,Black,None,None,Medium,...,1,None,UT,None,0,None,59044800,1,None,59044800
6,Cat,Grapeseed,Senior,Domestic Medium Hair,None,Female,Black,None,None,Medium,...,1,None,UT,None,0,None,59044793,1,None,59044793
7,Dog,Cookie,Young,Mixed Breed,Black Labrador Retriever,Female,Black,White / Cream,None,Medium,...,1,None,UT,None,0,Meet Cookie! This high energy pup loves to run...,59042189,4,None,59042189
8,Cat,Miss Lizzy,Young,Domestic Short Hair,None,Female,Calico,None,None,Medium,...,1,None,UT,None,0,"Hello humans, my name is Miss Lizzy! I am a qu...",59036436,1,None,59036436
9,Cat,Mike Wheeler,Baby,Domestic Short Hair,None,Male,Black,White,None,Small,...,1,None,UT,None,0,None,59035028,1,None,59035028


In [10]:
#df.to_csv('data/best_friends_animal_society/test.csv', index = False)

In [14]:
# Applying harmonization logic.
from petfinder_animal import petfinder_animal
fourth_animal = json_response['animals'][6]

harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(fourth_animal)

# Converting harmonized animal to dataframe.
print(harmonized_animal.as_kaggle_dictionary())
harmonized_animal_df = pd.DataFrame([harmonized_animal.as_kaggle_dictionary()])
harmonized_animal_df

{'Type': 2, 'Breed1': 0, 'Breed2': 0, 'Gender': 2, 'Color1': 1, 'Color2': 0, 'Color3': 0, 'MaturitySize': 2, 'FurLength': 0, 'Vaccinated': 1, 'Dewormed': 3, 'Sterilized': 1, 'Health': 0, 'State': 0, 'HasDescription': False, 'Age': 0, 'Quantity': 1, 'Fee': 0, 'VideoAmt': 0, 'PhotoAmt': 1, 'DescriptionLength': 0, 'nltk_negative_prob': 0, 'nltk_neutral_prob': 0, 'nltk_positive_prob': 0, 'nltk_compound_score': 0, 'AvgLaPlacianVariance': 233.1358013840664}


,Type,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,...,Quantity,Fee,VideoAmt,PhotoAmt,DescriptionLength,nltk_negative_prob,nltk_neutral_prob,nltk_positive_prob,nltk_compound_score,AvgLaPlacianVariance
0,2,0,0,2,1,0,0,2,0,1,...,1,0,0,1,0,0,0,0,0,233.135801


In [15]:
# Load Model with pickle.
model = pickle.load(open('models/random_forest_classifier_kaggle.pkl','rb'))

In [16]:
# Call Prediction on dataframe.
#harmonized_animal_df.columns
print(model.predict(harmonized_animal_df))

[2]


In [ ]:
print(harmonized_animal.Description)

In [ ]:
# Getting feedback from the animal:
from petfinder_feedback import petfinder_feedback

feedback = petfinder_feedback(harmonized_animal)
print(harmonized_animal.Url)
print(feedback.sentence_sentiments)
print(feedback.photo_blurriness)
print(feedback.sorted_negative_sentences)

In [ ]:
# Check for other animals.
for animal in json_response['animals']:
    harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(animal)
    print("Feedback:", harmonized_animal.Url)
    feedback = petfinder_feedback(harmonized_animal)
    print(feedback.sentence_sentiments)
    print(feedback.photo_blurriness)
    #print(feedback.top_negative_sentences)
    print(len(feedback.sorted_negative_sentences))
    for entry in feedback.sorted_negative_sentences:
        print(entry[0], ":", entry[1])
    print()
    print()

In [ ]:
#harmonized_animal.Photos[0]['small']
#res = requests.get(harmonized_animal.Photos[0]['small'], stream = True)
#with open("temp.jpg",'wb') as f:
#        shutil.copyfileobj(res.raw, f)
#urllib.urlretrieve(harmonized_animal.Photos[0]['small'], full_path)